# Tanzania Water Pump- Machine Learning Analysis

---

### Explore several ML classification algorithms 

Predict whether a pump is functional, functional needing repair, or non-functional using data from [Taarifa](http://taarifa.org/) and [Tanzania Ministry of Water](http://maji.go.tz/) based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A proper understanding of which water pumps are likely to fail could optimize maintenance operations and more reliably provide Tanzanian citizens with potable water.

This predictive modeling challenge comes from [DrivenData](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/), an organization that helps non-profits by hosting data science competitions for social impact. The competition has open licensing: "The data is available for use outside of DrivenData." The data was provided for a private Kaggle competition held as part of BloomTech's Data Science curriculum.

### Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, validation_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
pd.set_option('display.max_columns', None)

# I. Load and Clean Data
Using the information and understanding obtained from the EDA notebook, a 'wrangle()' function was written to perform the same data cleaning operations on both the training and testing data. This will also allow us to reproduce our problem with new unseen data.

In [4]:
def wrangle(feature_path, target_path=None):
    """
        This function loads and cleans data for feature matrix and target vector
        csv files. The cleaning tasks include:

            - Replace implicit nulls with NaN's
            - Convert datatypes
            - Remove unnecessary columns (duplicate, redundant, constant,
              mostly null)
            - Remove high-cardinality categorical features
        
        Parameters
        ----------
        feature_path (str): pathway to feature matrix csv file
        target_path (str): pathway to target vector csv file

        Returns
        -------
        DataFrame
    """

    if target_path:
        df = pd.merge(pd.read_csv(feature_path,
                                  parse_dates=['date_recorded']),
                      pd.read_csv(target_path)).set_index('id')

    else:
        df = pd.read_csv(feature_path,
                         parse_dates=['date_recorded'],
                         index_col='id')

    # Convert implicit nulls to nan's
    df['longitude'].replace(0, np.nan, inplace=True)
    df['latitude'].replace(-2e-08, np.nan, inplace=True)
    df['construction_year'].replace(0, np.nan, inplace=True)

    # Convert datatypes
    df['public_meeting'] = df['public_meeting'].astype('bool')
    df['permit'] = df['permit'].astype('bool')

    # Remove unnecessary columns
    df.drop(columns=['num_private',
                     'region_code',
                     'district_code',
                     'recorded_by',
                     'scheme_management',
                     'scheme_name',
                     'extraction_type_group',
                     'extraction_type_class',
                     'payment_type',
                     'quality_group',
                     'quantity_group',
                     'source',
                     'source_class',
                     'waterpoint_type_group'], inplace=True)

    # Remove HCCCs (columns with over 100 different categories)
    cutoff = 100
    drop_cols = [col for col in df.select_dtypes('object').columns 
                if df[col].nunique() > cutoff]
    df.drop(columns=drop_cols, inplace=True)

    # Create age feature
    df['pump_age'] = df['date_recorded'].dt.year - df['construction_year']
    df.drop(columns='date_recorded', inplace=True)


    return df
                     

In [5]:
# Load data
df = wrangle(feature_path='train_features.csv',
             target_path='train_labels.csv')

### Further EDA

In [6]:
# View first 5 rows of df and X_test
display(df.head())

,amount_tsh,gps_height,longitude,latitude,basin,region,population,public_meeting,permit,construction_year,extraction_type,management,management_group,payment,water_quality,quantity,source_type,waterpoint_type,status_group,pump_age
id,,,,,,,,,,,,,,,,,,,,
454,50.0,2092,35.426020,-4.227446,Internal,Manyara,160,True,True,1998.0,gravity,water board,user-group,pay per bucket,soft,insufficient,spring,communal standpipe,functional,15.0
510,0.0,0,35.510074,-5.724555,Internal,Dodoma,0,True,True,NaN,india mark ii,vwc,user-group,never pay,soft,enough,shallow well,hand pump,functional,NaN
14146,0.0,0,32.499866,-9.081222,Lake Rukwa,Mbeya,0,True,False,NaN,other,vwc,user-group,never pay,soft,enough,shallow well,other,non functional,NaN
47410,0.0,0,34.060484,-8.830208,Rufiji,Mbeya,0,True,True,NaN,gravity,vwc,user-group,pay monthly,soft,insufficient,river/lake,communal standpipe,non functional,NaN
1288,300.0,1023,37.032690,-6.040787,Wami / Ruvu,Morogoro,120,True,True,1997.0,other,vwc,user-group,pay when scheme fails,salty,enough,shallow well,other,non functional,14.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 454 to 23812
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   amount_tsh         47520 non-null  float64
 1   gps_height         47520 non-null  int64  
 2   longitude          46087 non-null  float64
 3   latitude           46087 non-null  float64
 4   basin              47520 non-null  object 
 5   region             47520 non-null  object 
 6   population         47520 non-null  int64  
 7   public_meeting     47520 non-null  bool   
 8   permit             47520 non-null  bool   
 9   construction_year  31017 non-null  float64
 10  extraction_type    47520 non-null  object 
 11  management         47520 non-null  object 
 12  management_group   47520 non-null  object 
 13  payment            47520 non-null  object 
 14  water_quality      47520 non-null  object 
 15  quantity           47520 non-null  object 
 16  source_type        4

In [8]:
df.isnull().sum()

amount_tsh               0
gps_height               0
longitude             1433
latitude              1433
basin                    0
region                   0
population               0
public_meeting           0
permit                   0
construction_year    16503
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source_type              0
waterpoint_type          0
status_group             0
pump_age             16503
dtype: int64

In [9]:
df.describe()

,amount_tsh,gps_height,longitude,latitude,population,construction_year,pump_age
count,47520.000000,47520.000000,46087.000000,46087.000000,47520.000000,31017.000000,31017.000000
mean,322.047573,668.745370,35.151330,-5.882390,179.528283,1996.819293,15.348615
std,3200.623244,692.972153,2.611187,2.808937,472.772997,12.481234,12.499078
min,0.000000,-63.000000,29.607122,-11.649440,0.000000,1960.000000,-7.000000
25%,0.000000,0.000000,33.275077,-8.633477,0.000000,1987.000000,4.000000
50%,0.000000,370.000000,35.010620,-5.164521,25.000000,2000.000000,13.000000
75%,20.000000,1320.000000,37.238584,-3.372232,213.000000,2008.000000,25.000000
max,350000.000000,2770.000000,40.345193,-0.998464,30500.000000,2013.000000,53.000000


We still have many null values in our columns. To ensure that our training and testing datasets undergo the same pre-processing steps, we will impute these values in a pipeline.

# II. Split Data

In [10]:
# Create feature matrix and target vector for training data
target = 'status_group'

y = df[target]
X = df.drop(columns=target)

In [11]:
# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


In [12]:
# Sanity check
display(X_train.shape, y_train.shape, X_val.shape, y_val.shape)


(38016, 19)

(38016,)

(9504, 19)

(9504,)

# III. Establish Baseline
We will use the most-frequent class as our baseline. Our model must outperform our baseline in order for our predictions to have any merit.

In [13]:
# Baseline accuracy score
print('Baseline accuracy:', y.value_counts(normalize=True).max())

Baseline accuracy: 0.5429713804713805


# IV. Build and Train Model

### Decision Tree Classifier

In [48]:
clf_dt = make_pipeline(
    OrdinalEncoder(handle_unknown='use_encoded_value',
                   unknown_value=np.nan),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=33)
)

In [49]:
# Create parameter grid for cross validation
param_dist_dt = {
    'simpleimputer__strategy' : ['mean', 'median', 'most_frequent'],
    'decisiontreeclassifier__max_depth' : range(10, 30, 2),
    'decisiontreeclassifier__min_samples_split' : range(2, 11, 2)
}

In [54]:
# Create and cross validate models
model_dt = RandomizedSearchCV(
    clf_dt,
    param_distributions=param_dist_dt,
    error_score='raise',
    n_jobs=-1,
    cv=5,
    verbose=1
)

In [55]:
# Fit model
model_dt.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder(handle_unknown='use_encoded_value',
                                                             unknown_value=nan)),
                                             ('simpleimputer', SimpleImputer()),
                                             ('decisiontreeclassifier',
                                              DecisionTreeClassifier(random_state=33))]),
                   n_jobs=-1,
                   param_distributions={'decisiontreeclassifier__max_depth': range(10, 30, 2),
                                        'decisiontreeclassifier__min_samples_split': range(2, 11, 2),
                                        'simpleimputer__strategy': ['mean',
                                                                    'median',
                                                                    'most_frequent']},
                   verbose=1)

In [56]:
print(f"Decision Tree Accuracy: {model_dt.best_score_}")
print(f"Decision Tree Parameters: {model_dt.best_params_}")

Decision Tree Accuracy: 0.7073071394130462
Decision Tree Parameters: {'simpleimputer__strategy': 'mean', 'decisiontreeclassifier__min_samples_split': 8, 'decisiontreeclassifier__max_depth': 10}


### Random Forest Classifier

In [36]:
# Create pipeline for pre-processing data
clf_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1, random_state=33)
)

# V. Check Evaluation Metrics

Compare with baseline

### Decision Tree Classifier

In [ ]:
cv_scores_dt = cross_val_score(clf_dt, X_train, y_train, cv=5, n_jobs=-1)
cv_scores_rf = cross_val_score(clf_rf, X_train, y_train, cv=5, n_jobs=-1)

/Users/erincostolo/Documents/Unit_2_Kaggle_Challenge/U2-ML/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/erincostolo/Documents/Unit_2_Kaggle_Challenge/U2-ML/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/erincostolo/Documents/Unit_2_Kaggle_Challenge/U2-ML/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 108, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/Users/erincostolo/Documents/Unit_2_Kaggle_Challenge/U2-ML/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/erincostolo/Documents/Unit_2_Kaggle_Challenge/U2-ML/lib/python3.8/site-packages/sklearn/pipeline.py", l

### Random Forest Classifier

# VI. Tune Model


# VII. Communicate Results